In [3]:
import pandas as pd
import numpy as np
import pathlib
from main import BASEPATH, IMAGESPATH, IMAGEFILENAME, BLOCKFILENAME


In [5]:
#   INITIAL DATA PREPARATION CELL
trainPath = BASEPATH + IMAGESPATH
#print(trainPath) # C:\Users\longw\Desktop\G Drive\Praca_inżynierska\Engineering_Thesis\backups\images\
withinImages= pathlib.Path(trainPath).glob('*')
dataPaths = [file for file in withinImages if file.is_dir()]
newestDir = str(dataPaths[-1]) + '\\'
#print(newestDir) # C:\Users\longw\Desktop\G Drive\Praca_inżynierska\Engineering_Thesis\backups\images\220117_21214749\
dfImages = pd.read_csv(newestDir + IMAGEFILENAME, header=None, names=['R','G','B'])
dfBlocks = pd.read_csv(newestDir + BLOCKFILENAME, header=None, names=['R','G','B'])
# changing the index, as the last index number will be equal to:
# the total number of images, and to the name of the image ({number}.png)
dfImages.index += 1
dfBlocks.index += 1
# deleting unnecessary data if the image's edges were rounded
dfImages = dfImages[0:len(dfBlocks)]

      R   G    B
1     5  21   83
2     5  21   83
3     5  21   83
4     5  21   84
5     5  21   83
6     5  21   83
7     5  21   83
8     5  21   83
9     5  21   83
10    0   1    5
11    6  28   68
12    3  12   56
13    3  12   56
14    5  12   57
15   47  13   93
16    7  12   59
17    3  12   56
18    3  12   56
19    8  27   67
20    1   2    5
21   10  48   72
22   13  42   74
23    6  15   40
24   82   6   98
25  192  11  201
26   92   7  108
27   13  13   45
28   26  49   89
29   15  61   84
30    0   3    4
31    9  43   76
32   15  46   99
33   24  45  103
34   80  40  136
35  152  42  196
36   90  41  144
37   27  44  104
38   19  45  107
39   12  45  106
40    0   2    5
41    9  48  108
42   12  45  104
43   15  43  105
44   49  43  123
45   73  40  139
